In [1]:
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import date
import json

In [2]:
import pandas as pd
import os
import datetime

In [3]:
now = datetime.datetime.now()

In [4]:
#Verifica diretório Atual 
os.getcwd() 

'C:\\Users\\filip\\OneDrive\\Documentos\\Python'

In [5]:
#Configura Diretório
os.chdir('C:\\Users\\filip\\OneDrive\\Documentos\\Python\\Arquivos') 

In [20]:
#Trata arquivo Bruto
dfFinal = pd.read_csv("consCodUOL.csv", sep = ";")

dfFinal.drop(dfFinal.columns[[0,1,4]], axis=1, inplace = True)

dfConsolidadp = pd.read_csv("CONSOLIDADO.csv", sep = ";")

dfFinal = dfFinal.merge(dfConsolidadp.loc[:,["cvm", "ano", "QtdAcoes", "Volume"]], on = ["cvm", "ano"], how = "inner")

for c in ['1', '1.01.01', '1.01.02', '1.02.01.01',
       '1.02.01.02', '1.02.01.03', '2.01.04', '2.01.02', '2.02.01', '3.05',
       'QtdAcoes']:
    dfFinal[c] = pd.to_numeric(dfFinal[c].str.replace(".", ""))

dfFinal["open"] = pd.to_numeric(dfFinal["open"].str.replace(".", "").str.replace(",", "."))
dfFinal["close"] = pd.to_numeric(dfFinal["close"].str.replace(".", "").str.replace(",", "."))

dfFinal["EBIT"] = dfFinal["3.05"]
dfFinal["ROIC"] = dfFinal["EBIT"] / (dfFinal["1"] - dfFinal["2.01.02"] - dfFinal["1.01.01"])
dfFinal["EV"] = dfFinal["QtdAcoes"] * dfFinal["open"]
dfFinal["DVIDALIQUIDA"] = dfFinal["2.01.04"] + dfFinal["2.02.01"] - dfFinal["1.01.01"] - dfFinal["1.01.02"] - dfFinal["1.02.01.01"] - dfFinal["1.02.01.02"] - dfFinal["1.02.01.03"]

#Exporta o Excel com a formula mágica
dfFinal.to_excel("dados_bolsa_full_"+now.strftime("%Y-%m-%d")+".xlsx")

In [21]:
#Importa tabela de Ticker que deve estar no diretório de Trabalho
dados_bolsa = pd.read_excel('BolsaConsolidadoSaneadoUOL.xlsx', sheet_name='Planilha1')
dados_bolsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 10 columns):
codigo          1672 non-null object
ano             1672 non-null int64
open            1672 non-null float64
close           1672 non-null float64
QtdAcoes        1672 non-null int64
Volume          1666 non-null float64
EBIT            1672 non-null int64
ROIC            1672 non-null float64
EV              1672 non-null float64
DVIDALIQUIDA    55 non-null float64
dtypes: float64(6), int64(3), object(1)
memory usage: 130.7+ KB


In [22]:
###Calcula Indicadores
dados_bolsa.DVIDALIQUIDA = dados_bolsa.DVIDALIQUIDA.fillna(0)

#EV + Divida Liquida
dados_bolsa['EV_DIVIDAS'] = dados_bolsa['EV']+dados_bolsa['DVIDALIQUIDA']

#EV/EBIT
dados_bolsa['EV/EBIT'] = dados_bolsa['EV_DIVIDAS']/dados_bolsa['EBIT']

index_table = dados_bolsa[['codigo','ano','Volume','close','QtdAcoes','ROIC','EV','EV/EBIT']]

index_table.columns = ['TICKER','ANO','VOL_MEDIO','COTACAO','NUM_ACOES','ROIC','EV','EV/EBIT']

index_table = index_table.sort_values(by=['ANO','TICKER'],ascending=True) 

In [25]:
########## Início do Programa ########## 
#Variáveis utilizadas
Cont1 = 0
Volume = 50000
QtdAcoes = 5
AnoInicio = 2009
AnoAtual = 2018
Ano = AnoInicio-1
year_stocks = []
my_stocks = []
range_stocks = []

while Ano != AnoAtual:
    Ano += 1
    Cont1 += 1
    print(" ")
    print(">>>>>>>>>>>>>>>Ano: "+str(Ano))
    
    #Busca dados para o Ano em questão
    year_stocks = index_table[index_table['ANO'] == Ano]
    
    #Verifica se o Ano possui valores
    if len(year_stocks)==0:
        print("--Nao existe empresas para este ano")
    else:    
        #Saneamento
        year_stocks = year_stocks.query("ROIC > 0")

        #Volume maior do que Definido nos parâmetros
        year_stocks = year_stocks[year_stocks['VOL_MEDIO'] > Volume]
        
        ###Begin - Retira Empresas repetidas, escolhendo a que possui menor cotacao - Begin
        year_stocks = year_stocks.sort_values('TICKER',ascending=True)

        year_stocks['TICKER_LETRAS'] = year_stocks['TICKER'].str.slice(0, 4, 1)

        year_stocks['MANTER'] = 'Y'

        index0 = 0
        for index0, row0 in year_stocks.iterrows():
            TickerMenorCotacao = year_stocks.loc[index0,'TICKER']
            TickerNiv0 = ""
            TickerNiv1 = ""
            MenorCotacao = year_stocks.loc[index0,'COTACAO']
            index1 = 0
            for index1, row1 in year_stocks.iterrows():
                TickerNiv0 = year_stocks.loc[index0,'TICKER']
                TickerNiv1 = year_stocks.loc[index1,'TICKER']

                if (TickerNiv0 != TickerNiv1) & (TickerNiv0[0:4] == TickerNiv1[0:4]):
                    if MenorCotacao > year_stocks.loc[index1,'COTACAO']:
                        TickerMenorCotacao = TickerNiv1
                        MenorCotacao = year_stocks.loc[index1,'COTACAO']
                    else:
                        year_stocks.loc[index1,'MANTER'] = 'N'
                    #end if
                #end if
            #end for
        #end for

        #Retira registros que nao devem ser mantidos
        year_stocks = year_stocks[year_stocks['MANTER'] == 'Y']

        ###End - Retira Empresas repetidas, escolhendo a que possui menor cotacao - End
        
        #Ordenando por EV/EBIT
        year_stocks = year_stocks.sort_values('EV/EBIT',ascending=True)

        #Rankeando EV/EBIT
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'ORD_EVEBIT'] = count

        #Ordenando por ROIC
        year_stocks = year_stocks.sort_values('ROIC',ascending=False) 

        #Rankeando ROIC
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'ORD_ROIC'] = count

        #Somando os 2 rankeamentos
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'RANKING'] = year_stocks.loc[index,'ORD_EVEBIT'] + year_stocks.loc[index,'ORD_ROIC']

        #Ordena tabela de rankeamento
        year_stocks = year_stocks.sort_values('RANKING',ascending=True)

        #Guarda quantidade de ações selecionadas do ano em questão
        range_stocks = year_stocks.head(QtdAcoes)[['TICKER','ANO','ROIC','EV','EV/EBIT','RANKING','NUM_ACOES','COTACAO']]
        range_stocks['NUM_ACOES_FIM_ANO'] = 0
        range_stocks['COTACAO_FIM_ANO'] = 0
        range_stocks['PRECO_INICIO_ANO'] = 0
        range_stocks['PRECO_FIM_ANO'] = 0
        range_stocks['STATUS'] = ''

        #Ordena tabela de Range
        range_stocks = range_stocks.sort_values('RANKING',ascending=True)

        #Guarda minhas ações do primeiro ano analisado
        if len(my_stocks) == 0:
            print(">> Ano Inicial")
            my_stocks = range_stocks
            my_stocks['PRECO_INICIO_ANO'] = my_stocks['EV']
            my_stocks['STATUS'] = 'COMPRADO'        
        else:                
            index = 0
            for index, row in my_stocks.iterrows():
                    
                #Considerar somente as acoes compradas do Ano Anterior da minha carteira 
                if (my_stocks.loc[index,'STATUS'] == 'COMPRADO')&(my_stocks.loc[index,'ANO'] == Ano-1):
                    print(">> Meu Ticker TICKER : "+my_stocks.loc[index,'TICKER']+". Ano: "+str(my_stocks.loc[index,'ANO'])+". Status: "+my_stocks.loc[index,'STATUS'])

                    #Adiciona Preço do Final do ano para o ano anterior           
                    price_table = year_stocks[year_stocks['TICKER'] == my_stocks.loc[index,'TICKER']]
                    index2 = 0
                    for index2, row2 in price_table.iterrows():
                        print("--- TICKER Atualizando Preco: "+my_stocks.loc[index,'TICKER']+" - TICKER Price Table: "+price_table.loc[index2, 'TICKER'])

                        my_stocks.loc[index,'COTACAO_FIM_ANO'] = price_table.loc[index2, 'COTACAO']
                        my_stocks.loc[index,'NUM_ACOES_FIM_ANO'] = price_table.loc[index2, 'NUM_ACOES']
                        my_stocks.loc[index,'PRECO_FIM_ANO'] = price_table.loc[index2, 'EV'] 
                    #end for

                    #Verifica se minhas acoes aparecem no Ano Atual 
                    remain_stocks = range_stocks[range_stocks['TICKER'] == my_stocks.loc[index,'TICKER']]

                    #Atualiza Status para vendido caso nao tenha que permanecer em carteira      
                    if len(remain_stocks) == 0:
                        my_stocks.loc[index,'STATUS'] = 'VENDIDO'
                        print("xxxxxxxx Acao vendida: "+my_stocks.loc[index,'TICKER'])                      
                    else:
                        #Atualiza preco inicial e status no ano seguinte
                        remain_stocks['PRECO_INICIO_ANO'] = remain_stocks['EV']
                        remain_stocks['STATUS'] = 'COMPRADO'

                        print(">>>>>>>> Acao remanescente: "+my_stocks.loc[index,'TICKER'])
                        my_stocks = my_stocks.append(remain_stocks)

                        #Apaga do range as novas acoes que ja comprei
                        range_stocks  = range_stocks[range_stocks.TICKER != my_stocks.loc[index,'TICKER']]                  
                    #end if
                #end if
            #end for

            print("")
            print("--- Verifica quantas acoes faltam comprar")

            #Ordena tabela Range pelo Ranking
            range_stocks = range_stocks.sort_values('RANKING',ascending=True)

            #Verifica quantas acoes faltam comprar       
            bought_stocks_year = my_stocks[(my_stocks['STATUS'] == 'COMPRADO')&(my_stocks['ANO'] == Ano)]         
            TargetStockQty = QtdAcoes - len(bought_stocks_year)

            if TargetStockQty>0:
                print(">>Quantidade Faltante: "+str(TargetStockQty))
                new_stocks = range_stocks.head(TargetStockQty)

                #Atualiza preco inicial e status
                new_stocks['PRECO_INICIO_ANO'] = new_stocks['EV']
                new_stocks['STATUS'] = 'COMPRADO'

                #Adiciona Novas Ações
                my_stocks = my_stocks.append(new_stocks)
            #end if
        #end if
    #end while
    
    #Atualiza Precos de acoes que nao existem no ano seguinte para o mesmo do ano anterior e monta resultado por empresa         
index3 = 0
print("--- TICKER Atualizando Preco zerados")
for index3, row3 in my_stocks.iterrows():

    if my_stocks.loc[index3,'COTACAO_FIM_ANO'] ==0:
        my_stocks.loc[index3,'COTACAO_FIM_ANO'] = my_stocks.loc[index3,'COTACAO']

    if my_stocks.loc[index3,'NUM_ACOES_FIM_ANO'] ==0:                            
        my_stocks.loc[index3,'NUM_ACOES_FIM_ANO'] = my_stocks.loc[index3,'NUM_ACOES']

    if my_stocks.loc[index3,'PRECO_FIM_ANO'] ==0: 
        my_stocks.loc[index3,'PRECO_FIM_ANO'] = my_stocks.loc[index3, 'EV']
    
    #Resultado Utilizando a Cotacao - Analitico
    my_stocks['VALOR_RESULT_ANUAL_COT'] = my_stocks['COTACAO_FIM_ANO'] - my_stocks['COTACAO']
    my_stocks['PERC_RESULT_ANUAL_COT'] = (my_stocks['VALOR_RESULT_ANUAL_COT']*100)/my_stocks['COTACAO']
    
    #Resultado Utilizando o Enterprize Value - Analitico
    my_stocks['VALOR_RESULT_ANUAL_EV'] = my_stocks['PRECO_FIM_ANO'] - my_stocks['PRECO_INICIO_ANO']
    my_stocks['PERC_RESULT_ANUAL_EV'] = (my_stocks['VALOR_RESULT_ANUAL_EV']*100)/my_stocks['PRECO_INICIO_ANO']
#end for

print("--- Relatórios Resultado")
#Resultado Sintetico
#sintetic_result = my_stocks.groupby(['ANO'])['COTACAO', 'COTACAO_FIM_ANO'].sum()
sintetic_result = my_stocks.groupby(['ANO'])['COTACAO', 'COTACAO_FIM_ANO','PRECO_INICIO_ANO', 'PRECO_FIM_ANO'].sum()

#Resultado Utilizando a Cotacao - Sintetico
sintetic_result['VALOR_RESULT_ANUAL_COT'] = sintetic_result['COTACAO_FIM_ANO'] - sintetic_result['COTACAO']
sintetic_result['PERC_RESULT_ANUAL_COT'] = (sintetic_result['VALOR_RESULT_ANUAL_COT']*100)/ sintetic_result['COTACAO']

#Resultado Utilizando o Enterprize Value - Sintetico
sintetic_result['VALOR_RESULT_ANUAL_EV'] = sintetic_result['PRECO_FIM_ANO'] - sintetic_result['PRECO_INICIO_ANO']
sintetic_result['PERC_RESULT_ANUAL_EV'] = (sintetic_result['VALOR_RESULT_ANUAL_EV']*100)/ sintetic_result['PRECO_INICIO_ANO']

#Exporta o Excel Resultado Anual
sintetic_result.to_excel("resultado_sintetico_"+str(QtdAcoes)+"_acoes.xlsx")

#Resultado Analitico
#analytic_result = my_stocks[['TICKER','ANO','COTACAO','COTACAO_FIM_ANO','VALOR_RESULT_ANUAL_COT','PERC_RESULT_ANUAL_COT']]
analytic_result = my_stocks[['TICKER','ANO','PRECO_INICIO_ANO', 'PRECO_FIM_ANO','VALOR_RESULT_ANUAL_EV','PERC_RESULT_ANUAL_EV','COTACAO','COTACAO_FIM_ANO','VALOR_RESULT_ANUAL_COT','PERC_RESULT_ANUAL_COT']]


analytic_result.to_excel("resultado_analitico_"+str(QtdAcoes)+"_acoes.xlsx")

print("FINAL")

 
>>>>>>>>>>>>>>>Ano: 2009
--Nao existe empresas para este ano
 
>>>>>>>>>>>>>>>Ano: 2010
>> Ano Inicial
 
>>>>>>>>>>>>>>>Ano: 2011
>> Meu Ticker TICKER : VALE3. Ano: 2010. Status: COMPRADO
--- TICKER Atualizando Preco: VALE3 - TICKER Price Table: VALE3
>>>>>>>> Acao remanescente: VALE3
>> Meu Ticker TICKER : BRAP4. Ano: 2010. Status: COMPRADO
xxxxxxxx Acao vendida: BRAP4
>> Meu Ticker TICKER : RENT3. Ano: 2010. Status: COMPRADO
--- TICKER Atualizando Preco: RENT3 - TICKER Price Table: RENT3


C:\Users\filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


>>>>>>>> Acao remanescente: RENT3
>> Meu Ticker TICKER : BBRK3. Ano: 2010. Status: COMPRADO
--- TICKER Atualizando Preco: BBRK3 - TICKER Price Table: BBRK3
xxxxxxxx Acao vendida: BBRK3
>> Meu Ticker TICKER : RAPT3. Ano: 2010. Status: COMPRADO
--- TICKER Atualizando Preco: RAPT3 - TICKER Price Table: RAPT3
>>>>>>>> Acao remanescente: RAPT3

--- Verifica quantas acoes faltam comprar
>>Quantidade Faltante: 2
 
>>>>>>>>>>>>>>>Ano: 2012
>> Meu Ticker TICKER : VALE3. Ano: 2011. Status: COMPRADO
--- TICKER Atualizando Preco: VALE3 - TICKER Price Table: VALE3
xxxxxxxx Acao vendida: VALE3
>> Meu Ticker TICKER : RENT3. Ano: 2011. Status: COMPRADO
--- TICKER Atualizando Preco: RENT3 - TICKER Price Table: RENT3
xxxxxxxx Acao vendida: RENT3
>> Meu Ticker TICKER : RAPT3. Ano: 2011. Status: COMPRADO
--- TICKER Atualizando Preco: RAPT3 - TICKER Price Table: RAPT3
xxxxxxxx Acao vendida: RAPT3
>> Meu Ticker TICKER : HGTX3. Ano: 2011. Status: COMPRADO
--- TICKER Atualizando Preco: HGTX3 - TICKER Price Ta

In [26]:
my_stocks

,TICKER,ANO,ROIC,EV,EV/EBIT,RANKING,NUM_ACOES,COTACAO,NUM_ACOES_FIM_ANO,COTACAO_FIM_ANO,PRECO_INICIO_ANO,PRECO_FIM_ANO,STATUS,VALOR_RESULT_ANUAL_COT,PERC_RESULT_ANUAL_COT,VALOR_RESULT_ANUAL_EV,PERC_RESULT_ANUAL_EV
171,VALE3,2010,0.206983,2.762596e+08,6.830963,20.0,5365305,55.33,5365305,39.45,2.762596e+08,3.056614e+08,COMPRADO,-15.88,-28.700524,2.940187e+07,1.064284e+01
12,BRAP4,2010,0.188889,1.370228e+07,5.585024,22.0,349548,43.32,349548,43.32,1.370228e+07,1.370228e+07,VENDIDO,0.00,0.000000,0.000000e+00,0.000000e+00
139,RENT3,2010,0.196421,3.933306e+06,8.022554,26.0,201708,26.90,201708,25.60,3.933306e+06,5.544953e+06,COMPRADO,-1.30,-4.832714,1.611647e+06,4.097436e+01
4,BBRK3,2010,0.255365,1.074058e+06,9.579537,28.0,168877,9.55,179043,5.58,1.074058e+06,1.699118e+06,VENDIDO,-3.97,-41.570681,6.250604e+05,5.819616e+01
133,RAPT3,2010,0.178772,3.276470e+06,7.277358,30.0,243785,9.50,243785,7.50,3.276470e+06,2.333022e+06,COMPRADO,-2.00,-21.052632,-9.434479e+05,-2.879464e+01
335,VALE3,2011,0.235768,3.056614e+08,5.757699,19.0,5365305,39.45,5365305,42.28,3.056614e+08,2.194410e+08,VENDIDO,2.83,7.173638,-8.622045e+07,-2.820783e+01
305,RENT3,2011,0.212003,5.544953e+06,9.308188,30.0,201708,25.60,201708,37.50,5.544953e+06,5.423928e+06,VENDIDO,11.90,46.484375,-1.210248e+05,-2.182612e+00
300,RAPT3,2011,0.148738,2.333022e+06,5.074226,32.0,243785,7.50,243785,9.77,2.333022e+06,1.828388e+06,VENDIDO,2.27,30.266667,-5.046350e+05,-2.163009e+01
253,HGTX3,2011,0.465938,4.566234e+05,1.250619,6.0,16343,32.46,16422400000000000,41.99,4.566234e+05,5.664086e+17,VENDIDO,9.53,29.359211,5.664086e+17,1.240428e+14
233,ETER3,2011,0.192016,1.011350e+04,0.086218,14.0,895,8.90,895,8.10,1.011350e+04,8.072900e+03,COMPRADO,-0.80,-8.988764,-2.040600e+03,-2.017699e+01
